In [1]:
#Importerer bibliotek for å behandle data og plotte.
import pandas as pd
import requests
import datetime
import numpy as np
import folium
from folium.features import DivIcon
from pyproj import CRS
from pyproj import Transformer

In [2]:
klimapunkter = [
    ['Hoemstinden', 6957846, 97865],
    ['Storhaugen', 6965858, 83674],
    ['Steinhaugen', 6956308, 95853],
    ['Rekdalshesten', 6971911, 76202],
    ['Rekdal', 6973843, 78145]
]

In [3]:
prosjektnavn = 'Hoemstinden'

#Gi in kordinater for posisjon og start og sluttdato for dataserien.
lon = 6957846    #NORD
lat = 97865     #ØST

#OBS!! Datoer for vinddata oppgis lenger ned i arket (eksister ikkje i like lang periode)
startdato = '1958-01-01' 
sluttdato = '2020-12-31'

In [4]:
transformer = Transformer.from_crs(5973, 4326)
trans_x, trans_y =  transformer.transform(lat, lon)

In [5]:
#Lager kartvisning for å verifisere at koordinater er for riktig stad
m = folium.Map(location=[trans_x, trans_y], zoom_start=12) #Bruker transformerte koordinater (ikkej funne ut korleis bruke UTM med folium)
folium.Circle(
    radius=250,
    location=[trans_x, trans_y],
    fill=False,
).add_to(m)
#Legger til Norgeskart WMS
folium.raster_layers.WmsTileLayer(
    url='https://opencache.statkart.no/gatekeeper/gk/gk.open_gmaps?layers=topo4&zoom={z}&x={x}&y={y}',
    name='Norgeskart',
    fmt='image/png',
    layers='topo4',
    attr=u'<a href="http://www.kartverket.no/">Kartverket</a>',
    transparent=True,
    overlay=True,
    control=True,
    
).add_child(folium.Popup('My name is Circle')).add_to(m)

m

In [108]:
def nve_api(lat, lon, startdato, sluttdato, para):
    '''
    Henter data frå NVE api GridTimeSeries
    input er lat, lon, dato og parameter for type data. UTM 33
    output er verdien i ei liste, men verdi per dag, typ nedbør osv..
    '''
    api = 'http://h-web02.nve.no:8080/api/'
    url = api + '/GridTimeSeries/' + str(lat) + '/' + str(lon) + '/' + str(startdato) + '/' + str(sluttdato) + '/' + para + '.json'
    r = requests.get(url)

    regn = r.json()
    return regn

In [141]:
def klima_dataframe(lat, lon, startdato, sluttdato):
    """Funksjonen tar inn ei liste  med lister med klimaparameter og lager dataframe

    Args:
        lat (str): øst-vest koordinat (i UTM33)
        lon (str): nord-sør koordinat (i UTM33)
        startdato (str): startdato for dataserien som hentes ned
        sluttdato (str): sluttdato for dataserien som hentes ned

    Returns
        df (dataframe): Returnerer ei pandas dataframe
    """
    rr = nve_api(lat, lon, startdato, sluttdato, 'rr') #Nedbør døgn
    #print(rr)
    altitude = rr['Altitude']
    fsw = nve_api(lat, lon, startdato, sluttdato, 'fsw') #Nysnø døgn
    sdfsw3d = nve_api(lat, lon, startdato, sluttdato, 'sdfsw3d') #Nynsø 3 døgn
    sd = nve_api(lat, lon, startdato, sluttdato, 'sd') #Snødybde
    tm = nve_api(lat, lon, startdato, sluttdato, 'tm') #Døgntemperatur
    start = datetime.datetime(int(startdato[0:4]), int(startdato[5:7]), int(startdato[8:10]))#'1960-01-01' 
    end = datetime.datetime(int(sluttdato[0:4]), int(sluttdato[5:7]), int(sluttdato[8:10]))
    #Etablerer pandas dataframe frå rr liste
    df = pd.DataFrame(rr['Data'])
    #Lager kolonne med datoer
    df['dato'] = pd.date_range(start, end)
    #Gir nytt navn til rr kolonna
    df.rename({0 : rr['Theme']}, axis=1, inplace=True)
    #Setter datokolonna til indexkolonna, slik at det går an å bruke grouperfunksjon for å sortere på tidsserie
    df.set_index('dato', inplace=True)
    #Etablerer kolonner i dataframefor andre parameter
    df[fsw['Theme']] = fsw['Data']
    df[sd['Theme']] = sd['Data']
    df[sdfsw3d['Theme']] = sdfsw3d['Data']
    df[tm['Theme']] = tm['Data']
    df['rr3'] = df.rr.rolling(3).sum() #Summerer siste 3 døgn 
    df[df > 60000] = 0
    return df, altitude

def max_df(df):
    maxrrsd3 = df['sdfsw3d'].groupby(pd.Grouper(freq='Y')).max()  #3 døgns snømengde
    maxrr = df['rr'].groupby(pd.Grouper(freq='Y')).max()
    maxrr3 = df['rr3'].groupby(pd.Grouper(freq='Y')).max()
    maxsd = df['sd'].groupby(pd.Grouper(freq='Y')).max()
    maxrr_df = pd.concat ([maxrr, maxrr3, maxrrsd3, maxsd], axis=1)

    return maxrr_df

In [131]:
df, altitude = klima_dataframe(klimapunkter[0][2], klimapunkter[0][1], startdato, sluttdato)
print('Høgde for klimapunktet: ' + str(altitude) + ' moh.')

Høgde for klimapunktet: 747 moh.


In [111]:
df

,rr,fsw,sd,sdfsw3d,tm,rr3
dato,,,,,,
1958-01-01,1.2,1,35.0,13.7,-9.45,NaN
1958-01-02,0.0,0,34.7,6.2,-13.95,NaN
1958-01-03,5.9,6,39.8,6.7,-11.95,7.100000e+00
1958-01-04,0.0,0,39.2,5.7,-10.75,5.900000e+00
1958-01-05,0.0,0,38.6,5.7,-8.45,5.900000e+00
...,...,...,...,...,...,...
2020-12-27,0.0,0,63.4,6.1,-0.35,9.100000e+00
2020-12-28,0.0,0,61.2,0.5,2.25,9.000000e-01
2020-12-29,0.0,0,60.5,0.0,-0.05,-4.739542e-13


In [112]:
#df.to_csv(prosjektnavn + '_klimadata.csv')

In [113]:
vindstart = '2018-03-01'
vindslutt = '2021-06-30'

In [114]:
#Vinddata - finnes kun i ein begrensa tidsperiode som grid data!

def wind_dataframe(lat, lon, vindstart, vindslutt):
    """Funksjonen tar inn ei liste  med lister med vinddata fra NVE gridtimeseries og lager dataframe

    Args:
        lat (str): øst-vest koordinat (i UTM33)
        lon (str): nord-sør koordinat (i UTM33)
        startdato (str): startdato for dataserien som hentes ned
        sluttdato (str): sluttdato for dataserien som hentes ned

    Returns:
        dfw (dataframe): Returnerer ei pandas dataframe med dager med vind sammen med nedbør over 2 mm og middeltemperatur under 1 grad og windhastighet over 2 m/s
    """
    windDirection = nve_api(lat, lon, vindstart, vindslutt, 'windDirection10m24h06') #Vindretning for døgnet
    windSpeed = nve_api(lat, lon, vindstart, vindslutt, 'windSpeed10m24h06')
    rr_vind = nve_api(lat, lon, vindstart, vindslutt, 'rr')
    tm_vind = nve_api(lat, lon, vindstart, vindslutt, 'tm')

    #3 timersdata mot vind, fra 2018-03-01 til 2019-12-31
    windDirection3h = nve_api(lat, lon, vindstart, vindslutt, 'windDirection10m3h')
    windSpeed3h = nve_api(lat, lon, vindstart, vindslutt, 'windSpeed10m3h')
    rr3h_vind = nve_api(lat, lon, vindstart, vindslutt, 'rr3h')
    tm3h_vind = nve_api(lat, lon, vindstart, vindslutt, 'tm3h')
    
    startwind = datetime.datetime(int(vindstart[0:4]), int(vindstart[5:7]), int(vindstart[8:10]))
    endwind = datetime.datetime(int(vindslutt[0:4]), int(vindslutt[5:7]), int(vindslutt[8:10]))
    
    df = pd.DataFrame(windDirection['Data']) #Henter inn verdiar for vindretning
    df['dato'] = pd.date_range(startwind, endwind) #Lager til datoer som ikkje kjem automatisk frå NVE
    df.rename({0 : windDirection['Theme']}, axis=1, inplace=True) #Gir nytt navn til kolonne med windretning
    df.set_index('dato', inplace=True) #Setter dato som index i dataframe
    df[windSpeed['Theme']] = windSpeed['Data']
    df[rr_vind['Theme']] = rr_vind['Data']
    df[tm_vind['Theme']] = tm_vind['Data']
    
    #Lager egne dataframes for å databehandle
    dfw = df.copy() #Snø
    dfwx = df.copy() #Generell vind
    dfwxrr = df.copy() #Nedbør
    
    indexNames = dfw[dfw['windSpeed10m24h06'] <= 2].index
    dfw.drop(indexNames , inplace=True)
    indexNames = dfw[dfw['rr'] <= 2].index
    dfw.drop(indexNames , inplace=True)
    indexNames = dfw[dfw['tm'] >= 1].index
    dfw.drop(indexNames , inplace=True)

    #dfw[dfw > 60000] = 0
    indexNames = dfwx[dfwx['windDirection10m24h06'] >= 1000].index
    dfwx.drop(indexNames , inplace=True)
    indexNames = dfwx[dfwx['windSpeed10m24h06'] >= 1000].index
    dfwx.drop(indexNames , inplace=True)
   
    #Lager dataframe med verdier for nedbør over 1 mm
    
    indexNames = dfwxrr[dfwxrr['rr'] <= 1].index
    dfwxrr.drop(indexNames , inplace=True)

    dfw3 = pd.DataFrame(windDirection3h['Data'])
    dfw3['dato'] = pd.date_range(startwind, endwind, freq='3h') #Lager til tidspunkt kvar 3. time
    dfw3.rename({0 : windDirection3h['Theme']}, axis=1, inplace=True)
    dfw3.set_index('dato', inplace=True)
    dfw3[windSpeed3h['Theme']] = windSpeed3h['Data']
    #print(len(windSpeed3h['Data']))
    #print(tm3h['Data'])
    dfw3[rr3h_vind['Theme']] = rr3h_vind['Data']
    dfw3['tm3h'] = tm3h_vind['Data']
    dfw3[dfw3 > 60000] = 0
    dfw['retning'] = dfw['windDirection10m24h06']*45
    dfwx['retning'] = dfwx['windDirection10m24h06']*45
    dfwxrr['retning'] = dfwxrr['windDirection10m24h06']*45
    
    return dfw, dfwx, dfwxrr
    

In [115]:
dfw, dfwx, dfwxrr = wind_dataframe(lat, lon, vindstart, vindslutt)

In [116]:
#dfw.to_csv(prosjektnavn + '_vindretning_snø.csv')
#dfwx.to_csv(prosjektnavn + '_vindretning_generell.csv')
#dfwxrr.to_csv(prosjektnavn + '_vindretning_nedbør.csv')

In [142]:
#Ekstremverdier
df_max = max_df(df)
df_max

,rr,rr3,sdfsw3d,sd
dato,,,,
1958-12-31,39.0,67.7,30.5,41.1
1959-12-31,53.5,86.6,30.5,57.2
1960-12-31,37.1,88.7,29.1,38.3
1961-12-31,44.0,78.7,61.6,61.8
1962-12-31,63.6,92.0,43.0,58.6
...,...,...,...,...
2016-12-31,51.2,86.5,32.7,33.0
2017-12-31,53.1,99.9,24.4,20.7
2018-12-31,51.0,89.3,7.5,8.5


In [134]:
import csv

In [139]:
df['rr'].to_csv('Hoemstinden_ekstrem_rr.csv', index=False)

C:\Users\jan.aalbu\AppData\Local\Continuum\anaconda3\envs\R\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [143]:
for index, klimapunkt in enumerate(klimapunkter):
    df, altitude = klima_dataframe(klimapunkt[2], klimapunkt[1], startdato, sluttdato)
    df.to_csv(klimapunkt[0] + '_klimadata.csv')
    
    #OBS!! feil, skal vere årlig max
    df_max['rr'].to_csv(klimapunkt[0] + '_ekstrem_rr.csv', index=False)
    df_max['rr3'].to_csv(klimapunkt[0] + '_ekstrem_rr3.csv', index=False)
    df_max['sdfsw3d'].to_csv(klimapunkt[0] + '_ekstrem_sdfsw3d.csv', index=False)
    print(f'Høgde for klimapunktet {klimapunkt[0]}: ' + str(altitude) + ' moh.')
    dfw, dfwx, dfwxrr = wind_dataframe(klimapunkt[2], klimapunkt[1], vindstart, vindslutt)
    dfw.to_csv(klimapunkt[0] + '_vindretning_snø.csv')
    dfwx.to_csv(klimapunkt[0] + '_vindretning_generell.csv')
    dfwxrr.to_csv(klimapunkt[0] + '_vindretning_nedbør.csv')
    with open(klimapunkt[0] + '_høgde.csv', 'w', encoding='UTF8', newline='') as f:
        writer = csv.writer(f)
        # write the data
        writer.writerow([altitude])
    

C:\Users\jan.aalbu\AppData\Local\Continuum\anaconda3\envs\R\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  
C:\Users\jan.aalbu\AppData\Local\Continuum\anaconda3\envs\R\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  import sys
C:\Users\jan.aalbu\AppData\Local\Continuum\anaconda3\envs\R\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  


Høgde for klimapunktet Hoemstinden: 747 moh.
Høgde for klimapunktet Storhaugen: 672 moh.
Høgde for klimapunktet Steinhaugen: 15 moh.
Høgde for klimapunktet Rekdalshesten: 677 moh.
Høgde for klimapunktet Rekdal: 117 moh.
